# Heroes of Pymoli analysis

1. Male players are considerably more than Female players (4:1) ratio.
2. Players tend to make more purchases when they are in their late teens to twenties. Especially the early twenties have high chance of buying items.
3. The number of purchases made for Popular items are not so different from number of purchases made for Profitable items. That might imply that Increasing the price of popular items, would help the profit.

In [17]:
#Dependencies
import os
import pandas as pd

#inputfiles 
input_files = ['purchase_data.json','purchase_data2.json']
df_1 = pd.read_json('purchase_data.json')
df_2 = pd.read_json('purchase_data2.json')

#reading into pandas dataframe
df = pd.read_json('purchase_data.json')

#adding a purchase serial number to dataframe
df = df.reset_index()
df = df.rename(columns={"index":"Serial number"})
df.head()

,Serial number,Age,Gender,Item ID,Item Name,Price,SN
0,0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,3,21,Male,92,Final Critic,1.36,Pheusrical25
4,4,23,Male,63,Stormfury Mace,1.27,Aela59


# Player count

In [18]:
#Total number of players

number_of_players = len(df["SN"].unique())
pd.DataFrame({"Total players":[number_of_players]})

,Total players
0,573


# Purchasing analysis - Total

In [19]:
#Purchasing analysis (Total) - Calculations

unique_items_count = len(df["Item ID"].unique())

item_price_df = df.loc[:,["Item ID","Item Name","Price"]]

item_price_df = item_price_df.drop_duplicates()

item_price_df.reset_index(drop=True)

average_purchasing_price = item_price_df["Price"].mean()

total_purchases=len(df)

total_purchases

total_revenue = df["Price"].sum()

#Purchasing analysis (Total) - Result Data Frame

purchasing_analysis_total = pd.DataFrame({"Number of Unique Items":[unique_items_count],
                                          "Average Purchase Price":[average_purchasing_price],
                                          "Total Number of Purchases":[total_purchases],
                                          "Total Revenue":[total_revenue]})

purchasing_analysis_total["Average Purchase Price"] = purchasing_analysis_total["Average Purchase Price"].map('${:,.2f}'.format)
purchasing_analysis_total["Total Revenue"] = purchasing_analysis_total["Total Revenue"].map('${:,.2f}'.format)

purchasing_analysis_total



,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,$2.95,183,780,"$2,286.33"


# Gender Demographics

In [21]:
#Separating Players into a view

unique_users_df = df.loc[:,["Age","Gender","SN"]]
unique_users_df = unique_users_df.drop_duplicates()

#Gender Demographics

players_gender_df = pd.DataFrame(unique_users_df["Gender"].value_counts())
players_gender_df =players_gender_df.reset_index()

players_gender_df = players_gender_df.rename(columns={"index":"Gender","Gender":"Number of players"})

total_players = players_gender_df["Number of players"].sum()

players_gender_df["Percentage of players"] = (players_gender_df["Number of players"]/total_players)*100
players_gender_df["Percentage of players"] = players_gender_df["Percentage of players"].map('{:,.2f}%'.format)

players_gender_df

,Gender,Number of players,Percentage of players
0,Male,465,81.15%
1,Female,100,17.45%
2,Other / Non-Disclosed,8,1.40%


In [22]:
gender_group = df.groupby("Gender")

purchase_count_by_gender = pd.DataFrame(gender_group["Serial number"].count())

avg_purchase_price_by_gender = pd.DataFrame(gender_group["Price"].mean().round(2))

total_purchase_by_gender = pd.DataFrame(gender_group["Price"].sum())

# Normalized total - with median as norm
normalized_total_by_gender = pd.DataFrame(total_purchase_by_gender/df["Price"].median())

purchase_count_by_gender.reset_index(level='Gender',inplace=True)
avg_purchase_price_by_gender.reset_index(level='Gender',inplace=True)
total_purchase_by_gender.reset_index(level='Gender',inplace=True)
normalized_total_by_gender.reset_index(level='Gender',inplace=True)

In [23]:
#Purchasing analysis (Gender) - Result Data Frame

purchase_analysis_gender = purchase_count_by_gender

result_items = [avg_purchase_price_by_gender,total_purchase_by_gender,normalized_total_by_gender]

for res_df in result_items:
    purchase_analysis_gender = purchase_analysis_gender.merge(res_df,on="Gender")

purchase_analysis_gender = purchase_analysis_gender.rename(columns={"Serial number":"Purchase count",
                                                                    "Price_x":"Average Purchase price",
                                                                    "Price_y":"Total purchase",
                                                                    "Price":"Normalized total"})

# Purchasing analysis - Gender

In [24]:
# Formatting data

purchase_analysis_gender["Average Purchase price"] = purchase_analysis_gender["Average Purchase price"].map('${:,.2f}'.format)
purchase_analysis_gender["Total purchase"] = purchase_analysis_gender["Total purchase"].map('${:,.2f}'.format)
purchase_analysis_gender["Normalized total"] = purchase_analysis_gender["Normalized total"].map('${:,.2f}'.format)

purchase_analysis_gender

,Gender,Purchase count,Average Purchase price,Total purchase,Normalized total
0,Female,136,$2.82,$382.91,$132.95
1,Male,633,$2.95,"$1,867.68",$648.50
2,Other / Non-Disclosed,11,$3.25,$35.74,$12.41


# Purchasing analysis - Age

In [25]:
#Purchasing analysis (Age)

age_bins = [0,10,15,20,25,30,35,40,45,50]

age_categories = ["under 10","10 - 14","15 - 19","20 - 24","25 - 29","30 - 34","35 - 39","40 - 44","45 and over"]

age_df = df

age_df["Age category"] = pd.cut(age_df['Age'], age_bins, labels=age_categories,right=False)

In [26]:
age_category_group = age_df.groupby("Age category")

purchase_count_by_age = pd.DataFrame(age_category_group["Serial number"].count())

avg_purchase_price_by_age = pd.DataFrame(age_category_group["Price"].mean().round(2))

total_purchase_by_age = pd.DataFrame(age_category_group["Price"].sum())

# Normalized total - with median as norm
normalized_total_by_age = pd.DataFrame(total_purchase_by_age/df["Price"].median())

purchase_count_by_age.reset_index(level='Age category',inplace=True)
avg_purchase_price_by_age.reset_index(level='Age category',inplace=True)
total_purchase_by_age.reset_index(level='Age category',inplace=True)
normalized_total_by_age.reset_index(level='Age category',inplace=True)


In [27]:
#Purchasing analysis (Age) - Result Data Frame

purchase_analysis_age = purchase_count_by_age

result_items = [avg_purchase_price_by_age,total_purchase_by_age,normalized_total_by_age]

for res_df in result_items:
    purchase_analysis_age = purchase_analysis_age.merge(res_df,on="Age category")

purchase_analysis_age = purchase_analysis_age.rename(columns={"Serial number":"Purchase count",
                                                                    "Price_x":"Average Purchase price",
                                                                    "Price_y":"Total purchase",
                                                                    "Price":"Normalized total"})
purchase_analysis_age = purchase_analysis_age.round(2)

In [28]:
# Formatting data
purchase_analysis_age["Average Purchase price"] = purchase_analysis_age["Average Purchase price"].map('${:,.2f}'.format)
purchase_analysis_age["Total purchase"] = purchase_analysis_age["Total purchase"].map('${:,.2f}'.format)
purchase_analysis_age["Normalized total"] = purchase_analysis_age["Normalized total"].map('${:,.2f}'.format)

purchase_analysis_age

,Age category,Purchase count,Average Purchase price,Total purchase,Normalized total
0,under 10,28,$2.98,$83.46,$28.98
1,10 - 14,35,$2.77,$96.95,$33.66
2,15 - 19,133,$2.91,$386.42,$134.17
3,20 - 24,336,$2.91,$978.77,$339.85
4,25 - 29,125,$2.96,$370.33,$128.59
5,30 - 34,64,$3.08,$197.25,$68.49
6,35 - 39,42,$2.84,$119.40,$41.46
7,40 - 44,16,$3.19,$51.03,$17.72
8,45 and over,1,$2.72,$2.72,$0.94


# Top Spenders

In [29]:
# Top Spenders

groupby_player = df.groupby("SN")

spenders_df = pd.DataFrame(groupby_player["Price"].sum())
spenders_df["Purchase count"] = groupby_player["Price"].count()
spenders_df["Average purchase price"] = (groupby_player["Price"].sum()/groupby_player["Price"].count()).round(2)

spenders_df.reset_index(level='SN',inplace=True)

top_spenders = spenders_df.nlargest(5,"Price").reset_index(drop=True)

top_spenders = top_spenders.rename(columns={"SN":"Player name","Price":"Total purchase value"})

top_spenders["Average purchase price"] = top_spenders["Average purchase price"].map("${:.2f}".format)
top_spenders["Total purchase value"] = top_spenders["Total purchase value"].map("${:.2f}".format)

top_spenders

,Player name,Total purchase value,Purchase count,Average purchase price
0,Undirrala66,$17.06,5,$3.41
1,Saedue76,$13.56,4,$3.39
2,Mindimnya67,$12.74,4,$3.18
3,Haellysu29,$12.73,3,$4.24
4,Eoda93,$11.58,3,$3.86


# Popular Items

In [30]:
# Popular Items 
groupby_item = df.groupby("Item ID")

items_purchase_count_df = pd.DataFrame(groupby_item["Serial number"].count())

items_purchase_count_df.reset_index(level='Item ID',inplace=True)
items_purchase_count_df = items_purchase_count_df.rename(columns={"Serial number":"Number of purchases"})


items_purchase_count_df = items_purchase_count_df.merge(item_price_df,on="Item ID")
items_purchase_count_df["Total purchase value"] = items_purchase_count_df["Price"]*items_purchase_count_df["Number of purchases"]


In [31]:
popular_items = items_purchase_count_df.nlargest(5,"Number of purchases").reset_index(drop=True)

column_order = ['Item ID', 'Item Name', 'Price', 'Number of purchases',
       'Total purchase value']

popular_items = popular_items[column_order]
popular_items["Price"] = popular_items["Price"].map("${:.2f}".format)
popular_items["Total purchase value"] = popular_items["Total purchase value"].map("${:.2f}".format)
popular_items

,Item ID,Item Name,Price,Number of purchases,Total purchase value
0,39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
1,84,Arcane Gem,$2.23,11,$24.53
2,13,Serenity,$1.49,9,$13.41
3,31,Trickster,$2.07,9,$18.63
4,34,Retribution Axe,$4.14,9,$37.26


# Profitable Items

In [32]:
profitable_items = items_purchase_count_df.nlargest(5,"Total purchase value").reset_index(drop=True)

column_order = ['Item ID', 'Item Name', 'Price', 'Number of purchases',
       'Total purchase value']

profitable_items = profitable_items[column_order]

profitable_items["Price"] = profitable_items["Price"].map("${:.2f}".format)
profitable_items["Total purchase value"] = profitable_items["Total purchase value"].map("${:.2f}".format)

profitable_items

,Item ID,Item Name,Price,Number of purchases,Total purchase value
0,34,Retribution Axe,$4.14,9,$37.26
1,115,Spectral Diamond Doomblade,$4.25,7,$29.75
2,32,Orenmir,$4.95,6,$29.70
3,103,Singed Scalpel,$4.87,6,$29.22
4,107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
